# 4. Retrieve image data and save it locally
In this case, we will focus on the following methods.

* Retrieve M-JPEG data saved on the server side as binary data
* Save as an image file (.jpg)

## Scenario
In this scenario, use iOS application **Visual M2M Motion** to stream video data, and save the data as images.

## Preparation
Before starting this scenario, prepare the following.

- An edge that performs measurement
- A measurement uploaded with Visual M2M Motion on iPhone (which contains images)


### Data to be used
In this scenario, the following data needs to be prepared on the server side.

| Data item | Data name that appears in this scenario |
|:---|:---|
|Edge to register time series data|sdk_edge1|
|Measurement(\*)|measurement1|

(\*) Register time series data in the same way as **"1. Retrieve time series data and save as CSV"** using Visual M2M Motion.

### Create measurement data and upload time series data
Upload the data using **Visual M2M Motion**. After registering the measurement, use **Visual M2M Data Visualizer** to check that the new measurement has been created.

### (Note) Operation of Visual M2M Motion
1. Start Visual M2M Motion on iPhone and sign in as `sdk_edge1`.
2. Make the following settings in Settings > Video.
    1. Enable Streaming.
    2. Enable Save to server.
    3. Set Codec to JPEG.
3. Return to the Main screen and tap ▶ in the center of the screen.
4. Confirm that the measurement is started
5. Tap ■ to end the measurement

Confirm that the new measurement is displayed in "Stored Data" of Visual M2M Data Visualizer.

<img src="https://github.com/aptpod/intdash-py-sample-codes/blob/master/img/img1.png?raw=true">

### Import packages and create a client
For `url` given to ` intdash.Client`, specify the environment of the intdash server. For `edge_token`, specify the token issued for the edge you use.
(* Login with `username` and `password` is also possible, but it is recommended to use edge token for continued use.)

In [8]:
import pandas as pd

import intdash
from intdash import timeutils

# Create client
client = intdash.Client(
    url = "https://example.intdash.jp",
    edge_token = "your_token",
)

The preparations are complete.

## Retrieve the edge used for measurement

In [9]:
edges = client.edges.list(name='sdk_edge1')
sdk_edge1 = edges[0]

In [10]:
sdk_edge1.name

'sdk_edge1'

## Retrieve time series data
In this scenario, the `client.data_points` endpoint is used to retrieve the time series data.
Normally, specify ``labels`` to retrieve data using signal definitions, but in this case, use `id_queries` because signal definitions are not used.

* Normally, a ``data_id`` needs to be specified. But for JPEG data, it is not necessary to specify the ``data_id`` because the ``data_id`` for JPEG is fixed. For details, see [intdash.data.Jpeg](https://docs.intdash.jp/sdk/python/latest/en/data.html#intdash.intdash.data.JPEG) in the document.

In [11]:
dps = client.data_points.list(
    edge_name='sdk_edge1',
    start=timeutils.str2timestamp('2020-07-17 00:00:00+09:00'),
    end=timeutils.str2timestamp('2020-07-18 00:00:00+09:00'),
    id_queries=[
        intdash.IdQuery(
            data_type=intdash.DataType.jpeg,
            channel=1
        )
    ]
)

## Save the retrieved data as an image file
Save the retrieved binary data as it is as JPEG.
Here, as a sample, only the first data point of the retrieved time series data is saved.

In [20]:
data = dps[0]  #  Specifies first data.

# Define the file name as a timestamp.
with open(f'./{pd.Timestamp(data.time).value}.jpg', 'wb') as fout:
    fout.write(data.data_payload)

If you want to save all time series data, repeat the above.

In [22]:
import os 

save_dir='./images'

for d in dps:
    
    if not os.path.isdir(save_dir):
            os.makedirs(save_dir)
            
    # Define the file name as a timestamp.
    with open(f'{save_dir}/{pd.Timestamp(d.time).value}.jpg', 'wb') as fout:
        fout.write(d.data_payload)